This is an attempt to find the publication date of each publication.

Problem: too many publications that takes a big ammount of time to run. Also, it has JavaScript, so it needs to be run by a Selenium Driver. 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from botasaurus import *
import os
import time
import requests
from tqdm import tqdm
from config.config_jupyter import set_wd
set_wd()
from src.constants import zona_prop_url

In [2]:
df = pd.read_parquet("output/zonaprop.pkt")

In [ ]:
from selenium.common.exceptions import WebDriverException

@browser
def scrap_info_extra_zonaprop(driver: AntiDetectDriver, url: str):
    retries = 0
    while True:
        try:
            driver.get(url)
            publ_vis = driver.text_xpath('//*[@id="user-views"]/div/div/p')
            tipo = driver.text_xpath('//*[@id="article-container"]/h2')
            driver._enable_network() # Assuming this method is correctly defined in the driver
            return {'publ_vis': publ_vis, 'tipo': tipo}
        except WebDriverException as e:
            # print(f"An error occurred: {str(e)}. Retrying...")
            retries += 1
            # print("n_fallas", retries)


urls = df['Link'].tolist() # Convert the 'Link' column to a list
reintentos = 0
while True:
    try:
        results = scrap_info_extra_zonaprop(parallel=bt.calc_max_parallel_browsers,
                                            data=urls,
                                            block_resources=True,
                                            cache = True,
                                            keep_drivers_alive=False, 
                                            headless=True,
                                            )
    except:
        reintentos += 1
        print("Reintentos", reintentos)        

# Add the scraped data to the dataframe
df['publ_vis'] = [result['publ_vis'] for result in results]
df['tipo'] = [result['tipo'] for result in results]